# Проект: Retrieval-система по статьям с arXiv


## Постановка задачи

В проекте вам предстоит построить retrieval-систему и оценить её качество. У вас будет датасет с аннотациями статей с портала arXiv.org и тестовый набор запросов и правильных ответов к ним. Также выполнить профилирование частей модели.

## Критерии успеха
Целевая метрика — качество поисковой выдачи: Mean Reciprocal Rank (MRR@5).

Проект считается выполненным при **MRR@5 > 0.91**.

<div class="alert alert-secondary" style="background-color:#D9EEE1;color:black;">

## Описание данных
Дан архив с датасетом.

В нём есть метаданные статей `arxiv-metadata-s.json` и набор тестовых запросов для оценки качества поиска `test_sample.csv`.
- `test_sample.csv` имеет структуру:
    - `query` — запрос к поисковой системе,
    - `id` — ID статьи, отвечающей на запрос query (один запрос — один верный ответ),
    - `abstract` — аннотация статьи id.
- arxiv-metadata-s.json имеет структуру:
    - `id` — ID статьи,
    - `abstract` — аннотация статьи,
    - `title` — название статьи.

## Инициализация

### Импорт библиотек

In [17]:
import pandas as pd
import json
from pandas import json_normalize

# Исследовательский анализ (EDA)

Загрузим и зучим данные датасета.

База стататей для RAG:

In [2]:
with open("arxiv-metadata-s.json", "r", encoding="utf-8") as f:
    arxiv_metadata = json.load(f)

In [3]:
len(arxiv_metadata)

98213

In [4]:
arxiv_metadata[0]

{'id': '0704.0038',
 'submitter': 'Maxim A. Yurkin',
 'authors': 'Maxim A. Yurkin, Alfons G. Hoekstra',
 'title': 'The discrete dipole approximation: an overview and recent developments',
 'comments': '36 pages, 1 figure; added several corrections according to the\n  published erratum except for Eq.(5) (it was correct in the original paper)',
 'journal-ref': 'J.Quant.Spectrosc.Radiat.Transf. 106, 558-589 (2007); Erratum:\n  J.Quant.Spectrosc.Radiat.Transf. 171, 82-83 (2016)',
 'doi': '10.1016/j.jqsrt.2007.01.034 10.1016/j.jqsrt.2015.11.025',
 'report-no': None,
 'categories': 'physics.optics physics.comp-ph',
 'license': 'http://creativecommons.org/licenses/by-nc-nd/4.0/',
 'abstract': '  We present a review of the discrete dipole approximation (DDA), which is a\ngeneral method to simulate light scattering by arbitrarily shaped particles. We\nput the method in historical context and discuss recent developments, taking\nthe viewpoint of a general framework based on the integral equation

Тестовый датасет:

In [12]:
test_sample_df = pd.read_csv("test_sample.csv")

In [13]:
test_sample_df.head()

,id,abstract,query
0,2412.16732,A new platinate was recently discovered when...,What unique composition and decomposition beha...
1,nucl-th/9602019,The production cross sections of various fra...,How does the inclusion of statistical decay af...
2,2501.05500,This survey provides a comprehensive examina...,What are the core components of modern zero-kn...
3,2506.20892,A critical challenge for operating fusion burn...,How does impurity seeding affect the timing an...
4,2208.02031,"In this work, we present the first corpus fo...",What is the primary challenge of the newly dev...


In [14]:
test_sample_df.iloc[0]['query']

'What unique composition and decomposition behavior does the newly discovered platinate Nd10.67Pt4O24 exhibit under thermal conditions?'

In [ ]:
test_sample_df.iloc[0]['abstract']

"  A new platinate was recently discovered when Nd2O3 was explored as a platinum\ncapture material in the Ostwald process, formed by a direct reaction between\nPtO2(g) and Nd2O3. The crystal structure of this new platinate and its\ncomposition, Nd10.67Pt4O24 , are here reported for the first time. The compound\nis synthesized either by a direct reaction using PtO2(g) or by the citric acid\nchemical route. Based on 3-dimensional electron diffraction data and Rietveld\nrefinement of high-resolution synchrotron and neutron powder diffraction data,\nwe accurately describe its crystal structure in space group I41/a. The compound\nis structurally related to the Ln11-xSrxIr4O24 (Ln = La, Pr, Nd, Sm) phases\nwith a double-perovskite (A2BB'O6) like crystal structure and A-site\ncation-deficiency. Owing to the fixed oxidation state of Pt(IV), two of the\nfour Nd-sites are partly occupied to provide charge neutrality, with the Nd4\nsite taking a split position. On heating, Nd10.67Pt4O24 decompose

In [15]:
test_sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1000 non-null   object
 1   abstract  1000 non-null   object
 2   query     1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


Посмотрим, все ли тестовые данные представлены в RAG-датасете:

In [18]:
arxiv_metadata_df = json_normalize(arxiv_metadata)

In [20]:
arxiv_metadata_df.head(2)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0038,Maxim A. Yurkin,"Maxim A. Yurkin, Alfons G. Hoekstra",The discrete dipole approximation: an overview...,"36 pages, 1 figure; added several corrections ...","J.Quant.Spectrosc.Radiat.Transf. 106, 558-589 ...",10.1016/j.jqsrt.2007.01.034 10.1016/j.jqsrt.20...,None,physics.optics physics.comp-ph,http://creativecommons.org/licenses/by-nc-nd/4.0/,We present a review of the discrete dipole a...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2022-03-30,"[[Yurkin, Maxim A., ], [Hoekstra, Alfons G., ]]"
1,0704.0057,Philipp Werner,Philipp Werner and Andrew J. Millis,High-spin to low-spin and orbital polarization...,Published version,"Phys. Rev. Lett. 99, 126405 (2007)",10.1103/PhysRevLett.99.126405,None,cond-mat.str-el,None,We study the interplay of crystal field spli...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2009-11-13,"[[Werner, Philipp, ], [Millis, Andrew J., ]]"


In [29]:
merged_df = pd.merge (test_sample_df, arxiv_metadata_df, how='inner', on='id')
merged_df.shape[0]

1000

In [31]:
merged_df.head(1)

,id,abstract_x,query,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract_y,versions,update_date,authors_parsed
0,2412.16732,A new platinate was recently discovered when...,What unique composition and decomposition beha...,{\O}ystein Slagtern Fjellv{\aa}g,"{\O}ystein Slagtern Fjellv{\aa}g, Helmer Fjell...","Crystal structure of Nd10.67Pt4O24, a new neod...",None,None,None,None,cond-mat.mtrl-sci,http://creativecommons.org/licenses/by/4.0/,A new platinate was recently discovered when...,"[{'version': 'v1', 'created': 'Sat, 21 Dec 202...",2024-12-24,"[[Fjellvåg, Øystein Slagtern, ], [Fjellvåg, He..."


Да, все тестовые образцы нашлись в RAG-датасете, это хорошо.

In [33]:
merged_df[merged_df['abstract_x'] == merged_df['abstract_y']].shape[0]

1000

И их абстракты совпадают, что тоже хорошо, т. к. не создаёт неоднозначностей сравнения.

## Выводы по итогам EDA

- датасет для RAG
    - 98213 статей с arxiv
    - статьи доступны для скачивания с arxiv
- test_sample.csv
    - 1000 тестовых примеров
    - все тестовые образцы нашлись в RAG-датасете, их абстракты совпадают.
- задачу планируется решать так:
    - Построим эмбеддинги для всех абстрактов RAG-датасета, добавим их в KB вместе с текстами абстрактов. Будем использовать для этого `Qwen/Qwen3-Embedding-0.6B`.
    - применим также ранжирующую модель `Qwen/Qwen3-Reranker-0.6B`
    - для поиска будем использовать `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B`


# Реализация retrieval-системы

# Оценка качества системы

# Выводы